# Sales Outreach Agent

Sales outreach is one of the most practicle business use cases for LLM agents.
Companies need to reach out to potential clients in a variety of tones from formal to fun. In this example , well :

* Define multiple agents, each with a unique communication style 
* Prompt them to write cold emails using OpenAI's GPT models
* Stream the generated response in real time 
* (Optional) Send the email using SendGrid

In [8]:
from dotenv import load_dotenv
from agents import Agent , Runner, trace
from openai.types.responses import ResponseTextDeltaEvent


load_dotenv(override=True)



True

In [9]:
# Define agent persona with distinct tones

professional_prompt = (
    "You are a professional agent at TechLink,"
    "an online education company offering AI-Driven , carrer-focused learning programs. "
    "Your goal is to write serious, persuasive cold outreach emails targeted at professionals and educational institutions."
)


engaging_prompt = (
    "You are a witty, engaging sales agent at TechLink, "
    "an online education company offering AI-powered, upskilling programs for tech and business roles. "
    "Craft humorous, attention-grabbing cold emails that spark curiosity and encourage responses."
)

concise_prompt = (
    "You are a fast-moving, efficient sales agent at TechLink, "
    "an online education platform offering smart, AI-powered learning paths. "
    "Write short, compelling cold emails that quickly highlight value and drive action."
)

In [10]:
# create 3 disctinct agents

professional_agent = Agent(
    name="Professional Sales Agent",
    instructions=professional_prompt,
    model="gpt-4o-mini"
)

engaging_agent = Agent(
    name="Engaging Sales Agent",
    instructions=engaging_prompt,
    model="gpt-4o-mini"
)

concise_agent = Agent(
    name="Concise Sales Agent",
    instructions=concise_prompt,
    model="gpt-4o-mini"
)

In [11]:
# Stream and display email content from an agent
email_result = Runner.run_streamed(engaging_agent, input="Write a cold sales email")
async for event in email_result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: 🚀 Ready to Upgrade Your Skills? No Rocket Science Required!

Hi [Recipient's Name],

Ever tried to fix your Wi-Fi by yelling at it? Spoiler: It doesn’t work. But if you’re looking to enhance your skills instead—without the tech tantrums—I've got just the ticket!

At TechLink, we're serving up AI-powered upskilling programs that make learning feel more like a Netflix binge and less like a math exam. From leveling up your coding skills to mastering the mysteries of business strategy, we’ve got courses that even your pet goldfish would nod off during (no offense to goldfish, they’re wonderful creatures).

🧠 Imagine this: You, conquering workplace challenges with newfound prowess, while your colleagues are still trying to figure out Zoom filters. 

Curious to learn how we can help you level up? Hit reply, and let’s chat! I promise I'll keep the tech jargon to a minimum (unless you ask for it). 

Best,

[Your Name]  
TechLink | Level Up Your Skills  
[Your Contact Information]  

P